# Produce plots for noise validation

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
import nibabel
from scipy import stats
from scipy import signal

%matplotlib notebook

## Show examples of the simulated data

In [ ]:
subj = 'Participant_01_rest_run01'
real_path = '../../Validation/real_data/%s.nii' % subj
sim_path = '../../Validation/simulated_data/%s.nii.gz' % subj
mask_path = '../../Validation/masks/%s.nii.gz' % subj

print('May take a minute or two to load data...')
nii = nibabel.load(real_path)
real_data = nii.get_data()
nii = nibabel.load(sim_path)
sim_data = nii.get_data()
nii = nibabel.load(mask_path)
mask_data = nii.get_data()
print('Data has been loaded')

In [ ]:
def slice_data(data, ylabel):
    plt.subplot(1, 3, 1)
    plt.imshow(data[:, :, 14, 0].reshape(64, 64), cmap=plt.cm.gray)
    plt.tick_params(axis='both', which='both', left='off', right='off', bottom='off', top='off', labelleft='off', labelbottom='off')
    plt.ylabel(ylabel)
    plt.title('Axial')

    plt.subplot(1, 3, 2)
    plt.imshow(np.rot90(data[32, :, :, 0].reshape(64, 27)), cmap=plt.cm.gray)
    plt.axis('off')
    plt.title('Sagital')

    plt.subplot(1, 3, 3)
    plt.imshow(np.rot90(data[:, 32, :, 0].reshape(64, 27)), cmap=plt.cm.gray)
    plt.axis('off')
    plt.title('Coronal')

plt.figure()
slice_data(real_data, 'real')
plt.savefig('../../Validation/plots/simulated_spatial_real.eps', format='eps', dpi=100)

plt.figure()
slice_data(sim_data, 'simulated')
plt.savefig('../../Validation/plots/simulated_spatial_simulated.eps', format='eps', dpi=100)

In [ ]:
# Plot the time course of a voxel

plt.figure()
y_range = [675, 775]

plt.subplot(2, 2, 1)
plt.plot(real_data[32, 32, 14, :])
plt.ylim(y_range)
plt.title('Voxel 1')
plt.ylabel('Real\nActivity')
plt.xticks([])

plt.subplot(2, 2, 2)
plt.plot(real_data[24, 32, 14, :])
plt.ylim(y_range)
plt.title('Voxel 2')
plt.yticks([])
plt.xticks([])

plt.subplot(2, 2, 3)
plt.ylabel('Simulated\nActivity')
plt.plot(sim_data[32, 32, 14, :])
plt.ylim(y_range)
plt.xticks([])

plt.subplot(2, 2, 4)
plt.plot(sim_data[24, 32, 14, :])
plt.ylim(y_range)
plt.yticks([])
plt.xticks([])
plt.xlabel('Time point')

plt.savefig('../../Validation/plots/simulated_temporal_voxel.eps', format='eps', dpi=100)

In [ ]:
# Plot the power spectrum of a voxel
def voxel_fft(voxel, TR=2):
    
    # What is the cut off
    cutoff = 1/100
    
    # Perform high pass filtering
    voxel = highpass_filter(voxel, cutoff, 1 / TR)
    
    # Number of samplepoints
    N = len(voxel)

    # Get the steps in hertz
    steps = np.linspace(0.0, 1.0/(2.0 * TR), N/2)
    
    # Calculate the spectral power
    power = 2.0 / N * abs(np.fft.fft(voxel)[:len(voxel) // 2])
    
    # Return the results
    return steps, power

# Create the temporal filter
def highpass_filter(data, cutoff, fs, order=5):
    
    # Take into account the nyquist theorem
    cutoff = cutoff / (fs / 2)
    
    # Create the filter
    b, a = signal.butter(order, cutoff, btype='high', analog=False)
    
    # Apply the filter
    filtered_data = signal.filtfilt(b, a, data)
    
    # Return the output
    return filtered_data

# Get an example fft
example_fft = voxel_fft(real_data[32, 32, 14, :])

# Create the template for the fft
fft_real = np.zeros((len(example_fft[1]),))
fft_sim = np.zeros((len(example_fft[1]),))

# Where are there voxels in the brain
brain_coords = np.where(mask_data == 1)

# Take a random sample of brain voxels
num_voxels = 1000
voxel_idxs = np.random.randint(low=0, high=len(brain_coords[0]), size=(num_voxels,))

for voxel_counter in voxel_idxs:
    
    # Pull out the sum
    x = brain_coords[0][voxel_counter]
    y = brain_coords[1][voxel_counter]
    z = brain_coords[2][voxel_counter]
    
    fft_real += voxel_fft(real_data[x, y, z, :])[1]
    fft_sim += voxel_fft(sim_data[x, y, z, :])[1]

# Get the average
fft_real /= len(voxel_idxs) 
fft_sim /= len(voxel_idxs) 

plt.figure(figsize=(2,4))
y_range = [0, 2.5]
x_ticks = [0.0, 0.25]

plt.subplot(2, 1, 1)
plt.plot(example_fft[0], fft_real)
plt.ylim(y_range)
plt.xticks(x_ticks, x_ticks)
plt.ylabel('Real\nFrequency')

plt.subplot(2, 1, 2)
plt.plot(example_fft[0], fft_sim)
plt.ylim(y_range)
plt.xticks(x_ticks, x_ticks)
plt.ylabel('Simulated\nFrequency')
plt.xlabel('Frequency (Hz)')

plt.savefig('../../Validation/plots/simulated_spectral_voxel.eps', format='eps', dpi=100)


## Compute the run time of validation analyses

In [ ]:
timing_file = '../../Validation/simulation_timing.txt'
fid = open(timing_file)
text = fid.readlines()
fid.close()

# Pull out the timing info
duration_all = []

# Iterate through the text
for line in text:
    
    # What is the timing duration
    duration = float(line[line.find(': ') + 1:line.find('\n')])
    
    if line[line.find('matching_') + 9] == '1':
        duration_all += [duration]
        
# Identify the average and SD of the different analyses

print('Duration: %0.3f (SD=%0.3f, max=%0.3f)' % (np.mean(duration_all), np.std(duration_all), np.max(duration_all)))
       

## Perform the validation analyses for fmrisim

In [ ]:
# Take the output of the signal_fit function and plot the heatmap of results

# Identify file. paths
testing_noise_calc_path = '../../Validation/testing_noise_calc/'

# What is the file name
real_noise_dicts = np.load(testing_noise_calc_path + 'real_noise_dicts.npy')
simulated_noise_dicts = np.load(testing_noise_calc_path + 'simulated_noise_dicts.npy')
simulated_noise_dicts_means = np.load(testing_noise_calc_path + 'simulated_noise_dicts_means.npy')
simulated_noise_dicts_std = np.load(testing_noise_calc_path + 'simulated_noise_dicts_stds.npy')

In [ ]:
# Create a function to compare the different inputs
def plot_boxplot(*arg,
                 ):
    plt.figure(figsize=(4, len(arg) * 3))
    
    # Reshape and store the data

    for arg_number in list(range(0, len(arg))):
        var = arg[arg_number]
        var = var.reshape(np.prod(var.shape))

        # Add the data to the matrix
        if arg_number == 0:
            data = np.asarray([var])
        else:
            data = np.vstack([data, var])

    # What is the x value
    x_idxs = list(range(1, data.shape[0] + 1))
    
    plt.plot(x_idxs, data, c=(0.9,0.9,0.9))
    
    # What indexes are usable from the data
    useable_idxs = np.where(np.prod(~np.isnan(data), axis=0)==1)[0]
    
    # Plot the box plots
    plt.boxplot(np.transpose(data[:, useable_idxs]))
    
    # If there are two inputs, do a t test
    if len(arg) == 2:
        t, p = stats.ttest_rel(data[1, useable_idxs], data[0, useable_idxs])
        delta_mean = np.nanmean(np.subtract(data[1, useable_idxs], data[0, useable_idxs]))
        df = data.shape[1] - 1
        print('Test statistics:')
        print("M=%0.2f, t(%d)=%0.2f, p=%0.3f" % (delta_mean,df,t,p))
        

In [ ]:
# How many of the simulated values are in range?
def plot_title_fit(title_prefix, real_noise, simulated_noise, threshold=0.05):

    # Pool all of the proportional differences in simulated noise 
    values = []
    for resample in range(simulated_noise.shape[2]):
        values = np.append(values, (abs(simulated_noise[:, :, resample] - real_noise[:, :]) / (real_noise[:, :])).flatten())

    # What proportion of items are above the fit threshold
    prop = 1 - (np.sum(np.asarray(values) > threshold) / len(values))

    plt.title('%s: prop within threshold = %0.3f' % (title_prefix, prop))


In [ ]:
# Create all of the plots
def plot_parameter(parameter_type):

    if parameter_type == 'SNR':
        parameter_idx = 0
    elif parameter_type == 'SFNR':
        parameter_idx = 1        
    elif parameter_type == 'FWHM':
        parameter_idx = 2        
    elif parameter_type == 'AR':
        parameter_idx = 3         
    elif parameter_type == 'MA':
        parameter_idx = 4         
            
    # Make the box plot
    plot_boxplot(real_noise_dicts[:, :, parameter_idx], simulated_noise_dicts_means[:, :, parameter_idx])
    plot_title_fit(parameter_type, real_noise_dicts[:, :, parameter_idx], simulated_noise_dicts[:, :, parameter_idx, :])
    plt.xticks(np.asarray([1,2]), ['Real', 'Simulated'])
    plt.savefig('../../Validation/plots/%s_comparison.eps' % parameter_type, format='eps', dpi=100)
    ylims = plt.ylim()

    # Plot the difference from the real value as a proportion of the value of real. This should be below the fit threshold that was used (default is 10) 
    plt.figure()
    diff = real_noise_dicts[:, :, parameter_idx] - simulated_noise_dicts_means[:, :, parameter_idx]
    plt.hist(abs(diff).flatten() / abs(real_noise_dicts[:, :, parameter_idx]).flatten())
    plt.xlabel('Difference from real, proportion of real')

    # Plot the distribution of noise metrics
    stds = simulated_noise_dicts_std[:, :, parameter_idx]
    stds = stds.reshape(np.prod(stds.shape))
    plt.figure()
    bins = np.linspace(0, ylims[1], 100)
    useable_idxs = np.where(~np.isnan(stds))[0]
    plt.hist(stds[useable_idxs], bins);
    plt.xlabel(parameter_type)
    print('Mean variance')
    print(np.nanmean(stds ** 2))

In [ ]:
plot_parameter('SNR')

In [ ]:
plot_parameter('SFNR')

In [ ]:
plot_parameter('FWHM')

In [ ]:
plot_parameter('AR')